In [10]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re

In [11]:
import nltk
from nltk.corpus import stopwords 
# stop_words = set(stopwords.words('english'))

In [12]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [13]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12242\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\12242\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
# import nltk
# vocabulary = nltk.FreqDist()

# # This was done once I had already preprocessed the ingredients
# for ingredients in df['ingredients_parsed']:
#     str1 = ""
# #     print(ingredients)
#     for i in ingredients:
#         str1 += " "+i 
#     ingredients = str1.split()
#     vocabulary.update(ingredients)
    
# for word, frequency in vocabulary.most_common(1000):
#     print(f'{word};{frequency}')

In [15]:
# vocabulary['?']

In [16]:
# remove everything past a comma
# remove everything past a number
# remove everything past "to", "for", "cut", "with"
# remove "as", "required", "chopped", "medium", "fresh", "boiled", "peeled", "and", "finely", "a", "pinch", "slice", "crushed", "fillets", "sliced", "few", "strands", "browned", "loaf", "pieces", "into", "cubes", "garnishing", "tablespo", "drop", "shaving", "drained" 
# remove anything with "salt", "oil", "butter"
# separate as ingredient what is inside () or after /

In [17]:
import unidecode
import ast
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords 

def ingredient_cleaner(ingreds):
    lemmatizer = WordNetLemmatizer()
    words_to_remove = ["as", "required", "chopped", "medium", "fresh", "boiled", "peeled", "and", "finely", "a", "pinch",                    
                       "slice", "crushed", "fillet", "sliced", "few", "strand", "browned", "loaf", "piece", "into", "cube", 
                       "garnishing", "tablespo", "drop", "shaving", "drained", "grated", "gram", "soaked", "roughly", "blanched",
                        "scraped", "ice", "large", "halved", "washed", "seeded", "in", "deveined", "diced", "slivered", "shelled",
                       "quartered", "bulb", "overnight", "on", "pearl", "desiccated", "the", "at", "from", "cup", "one", "two", 
                       "three", "four", "five", "six", "seven", "eight", "nine", "ten", "plenty", "of", "", " "]
    
#     stop_words = set(stopwords.words('english'))
    
    if isinstance(ingreds, list):
        items = ingreds
    else:
        items = ast.literal_eval(ingreds)
        
    items = [unidecode.unidecode(word) for word in items]
    items = [word.replace("(", "") for word in items]
    items = [word.replace(")", "") for word in items]
    items = [word.replace("/", " ") for word in items]
    items = [word.replace("-", " ") for word in items]
    items = [word.replace("+", " ") for word in items]
    items = [word.replace("?", " ") for word in items]
    # remove punctuations
    # check against tuitiorial 
    items = [word.split(",")[0] for word in items]
    items = [re.split(r'(\d+)', word)[0] for word in items]
    items = [word.lower() for word in items]
    items = [word.split(" to")[0] for word in items]
    items = [word.split("for")[0] for word in items]
    items = [word.split("cut")[0] for word in items]
    items = [word.split(" with")[0] for word in items]
    items = [word.replace("  ", " ") for word in items]
    items = [word for word in items if word.find("oil") == -1]
    items = [word for word in items if word.find("salt") == -1]
    items = [word for word in items if word.find("butter") == -1]
    items = [word for word in items if word.find("sugar") == -1]
    items = [word for word in items if word.find("lemon") == -1]
    # lemmatizing words
    items_lemmatized = []
    for i in range(0, len(items)):
        to_lemma = items[i].split(' ')
        word_lemmatized = [lemmatizer.lemmatize(word) for word in to_lemma]
        word_cleaned = ' '.join([word for word in word_lemmatized if word not in words_to_remove])
        items_lemmatized.append(word_cleaned.lstrip().rstrip())
    items_lemmatized = [word for word in items_lemmatized if word != '']
#     items = listToString(items)
#     items = [lemmatizer.lemmatize(word) for word in list(items.split(" "))]
#     items = [word for word in items if word not in words_to_remove]
#     items = [word for word in items if word not in stop_words]
#     lammetize and change plurals from words_to_remove
#     items = listToString(items)
    return items_lemmatized   

In [18]:
if __name__ == "__main__":
    df = pd.read_csv('df_scrubbed.csv')
    df = df.dropna()
    df['ingredients_parsed'] = df['ingredients'].apply(lambda x: ingredient_cleaner(x))
    df = df.iloc[: , 1:]

    df.to_csv('df_parsed_2.csv', index=False)

In [19]:
df = pd.read_csv('df_parsed_2.csv')